# XGBoost vs LightGBM

In this notebook we collect the results from all the experiments and reports the comparative difference between XGBoost and LightGBM

In [ ]:
import matplotlib.pyplot as plt
import nbformat
import json
from toolz import pipe, juxt
import pandas as pd
import seaborn
%matplotlib inline 

We are going to read the results from the following notebooks

In [ ]:
notebooks = {
    'Airline':'01_airline.ipynb',
    'BCI': '02_BCI.ipynb',
    'Football': '03_football.ipynb',
    'Amazon': '04_PlanetKaggle.ipynb',
    'Fraud': '05_FraudDetection.ipynb'
}

In [ ]:
def read_notebook(notebook_name):
    with open(notebook_name) as f:
        return nbformat.read(f, as_version=4)

In [ ]:
def results_cell_from(nb):
    for cell in nb.cells:
        if cell['cell_type']=='code' and cell['source'].startswith('# Results'):
            return cell

In [ ]:
def extract_results_from(cell):
    return json.loads(cell['outputs'][0]['text'])

In [ ]:
def process_nb(notebook_name):
    return pipe(notebook_name,
                read_notebook,
                results_cell_from,
                extract_results_from)

Here we collect the results from all the exeperiment notebooks. The method simply searches the notebooks for a cell that starts with # Results. It then reads that cells output in as JSON.

In [ ]:
results = {nb_key:process_nb(nb_name) for nb_key, nb_name in notebooks.items()}

In [ ]:
results

We wish to compare LightGBM and XGBoost both in terms of performance as well as how long they took to train.

In [ ]:
def average_performance_diff(dataset):
    lgbm_series = pd.Series(dataset['lgbm']['performance'])
    return 100*((lgbm_series-pd.Series(dataset['xgb']['performance']))/lgbm_series).mean()

In [ ]:
def train_time_ratio(dataset):
    return dataset['xgb']['train_time']/dataset['lgbm']['train_time']

def test_time_ratio(dataset):
    return dataset['xgb']['test_time']/dataset['lgbm']['test_time']

In [ ]:
metrics = juxt(average_performance_diff, train_time_ratio, test_time_ratio)
res_per_dataset = {dataset_key:metrics(dataset) for dataset_key, dataset in results.items()}

In [ ]:
results_df = pd.DataFrame(res_per_dataset, index=['Perf. Difference(%)', 
                                                  'Train Time Ratio', 
                                                  'Test Time Ratio']).T

In [ ]:
results_df

In [ ]:
ax = results_df[['Train Time Ratio']].plot(kind='bar');
fig = ax.get_figure()
fig.savefig('train_time_ratio.svg', bbox_inches='tight')

In [ ]:
ax = results_df[['Perf. Difference(%)']].plot(kind='bar');
ax.legend(loc=(0.64,0.8))
fig = ax.get_figure()
fig.savefig('perf_diff.svg', bbox_inches='tight')

From the table as well as the plots below we can see that overall the difference in performance is quite small. LightGBM though is 2 to over 10 times quicker than XGBoost.